In [ ]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

nltk.download("stopwords")

def extract_key_terms_using_tfidf(text, max_features=None):
    # Define a vectorizer with custom parameters
    vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), max_features=max_features)
    
    # Fit and transform the text
    tfidf_matrix = vectorizer.fit_transform([text])
    
    # Create a dictionary to store words and their scores
    word2tfidf = defaultdict(float)
    for word, index in vectorizer.vocabulary_.items():
        word2tfidf[word] = tfidf_matrix[0, index]
    
    # Sort the dictionary by the scores in descending order and return the terms
    sorted_terms = sorted(word2tfidf.items(), key=lambda x: x[1], reverse=True)
    
    return ' '.join([term[0] for term in sorted_terms])





In [ ]:
import actions
import funcs_database
import funcs_supabase

# Gets terms and saves to Supabase for A GIVEN ALBUM


def get_field_value(item, field):
    """Returns the value associated with the given field in the item dictionary.
    If the field is not present or its value is None, returns an empty string."""
    return item.get(field, "") or ""

album_id = 110
media_list = funcs_database.get_media_by_album_id(album_id)

for item in media_list:

    # Get the terms from the item
    current_terms = item.get("terms", "")
    print("Current Terms:", current_terms)

    if current_terms:
        continue

    else:
        terms = " ".join(
            get_field_value(item, field) 
            for field in ["description", "caption", "input_phrase"]
            if get_field_value(item, field)  # This filters out empty strings
        )
        print(terms)
        
        key_terms = extract_key_terms_using_tfidf(terms)
        print("Key Terms:", key_terms)

        update = funcs_supabase.update_data( "media" , item["id"], {"terms": key_terms})
        



In [ ]:
import vector_search

album_id = 110
media_list = funcs_database.get_media_by_album_id(album_id)

for item in media_list:
    # Get the terms from the item
    current_terms = item.get("terms", "")
    new_data = {
        "content": item.get("terms"),
        "media_table_id": item.get("id"),
        "context": "media_terms",
        "embedding": None
    }

    # Search Supabase for a row with these terms and a non-null embedding
    supabase_search_results = funcs_supabase.select_data_with_conditions("vectors", [
        {"column": "content", "operator": "eq", "value": current_terms}
    ])

    embedding_found = False
    for res in supabase_search_results:
        embedding = res.get("embedding")

        if embedding:  # if a valid embedding is found, use it and break
            embedding_found = True
            new_data["embedding"] = embedding

            # Before inserting, check for duplicates
            existing_record = funcs_supabase.select_data_with_conditions("vectors", [
                {"column": "content", "operator": "eq", "value": new_data["content"]},
                {"column": "media_table_id", "operator": "eq", "value": new_data["media_table_id"]}
            ])

            if not existing_record:
                funcs_supabase.insert_data("vectors", new_data)
            elif not existing_record[0].get("embedding"): # check if the embedding is missing
                funcs_supabase.update_data("vectors", existing_record[0]["id"], {"embedding": embedding})
                print("Updated missing embedding for existing record.")
            else:
                print("Record with given content, media_table_id, and embedding already exists.")

            break

    # If no valid embedding was found in the table, generate a new one
    if not embedding_found:
        embedding = vector_search.get_embeddings(current_terms)
        new_data["embedding"] = embedding

        # Before inserting, check for duplicates
        existing_record = funcs_supabase.select_data_with_conditions("vectors", [
            {"column": "content", "operator": "eq", "value": new_data["content"]},
            {"column": "media_table_id", "operator": "eq", "value": new_data["media_table_id"]}
        ])

        if not existing_record:
            funcs_supabase.insert_data("vectors", new_data)
        elif not existing_record[0].get("embedding"): # check if the embedding is missing
            funcs_supabase.update_data("vectors", existing_record[0]["id"], {"embedding": embedding})
            print("Updated missing embedding for existing record.")
        else:
            print("Record with given content, media_table_id, and embedding already exists.")


In [ ]:
# filters_dict = {"source_type": "sd", "type": "video"}
filters_dict = {}

In [ ]:
album_results_filtered = []

# Loop through each dictionary in album_results
for result in album_results:
    matches_all_filters = True  # Assume the current result matches all filters
    
    # Loop through each key-value pair in filters_dict
    for key, value in filters_dict.items():
        # If the key-value pair doesn't match, set matches_all_filters to False
        if result.get(key) != value:
            matches_all_filters = False
            break  # Exit the inner loop as we found a mismatch
    
    # If after checking all filters we still have matches_all_filters as True, append the result to the filtered list
    if matches_all_filters:
        album_results_filtered.append(result)


In [ ]:
album_results_filtered_id_list = []
for item in album_results_filtered:
    print(item)
    album_results_filtered_id_list.append(item['id'])

print(album_results_filtered_id_list)

Check if search is selected.

Check if all the media items in this album have a vector to search against. 

For the ones without vectors, give them a vector. 

k



In [ ]:
import psycopg2
import os
import psycopg2.extras  # Import the extras
from dotenv import load_dotenv

def get_media_ids_not_in_vectors(media_ids):
    # Load environment variables
    load_dotenv()

    # Get database connection string from environment variables
    SUPABASE_DB_FULL_URL = os.getenv("SUPABASE_DB_FULL_URL")

    # Initialize connection and cursor
    conn = psycopg2.connect(SUPABASE_DB_FULL_URL)
    cursor = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

    # SQL query
    sql_query = """
    SELECT id
    FROM media
    WHERE id NOT IN (
        SELECT media_table_id
        FROM vectors
        WHERE media_table_id = ANY(%s)
    )
    AND id = ANY(%s);
    """

    missing_ids = []

    try:
        # Execute the query
        cursor.execute(sql_query, (media_ids, media_ids))

        # Fetch all rows and store the IDs in a list
        rows = cursor.fetchall()
        for row in rows:
            missing_ids.append(row["id"])

    except Exception as e:
        print(f"An error occurred while executing the SQL query: {e}")

    finally:
        # Close the cursor and the connection
        cursor.close()
        conn.close()

    return missing_ids


In [ ]:
rr= get_media_ids_not_in_vectors(album_results_filtered_id_list)
print(rr)

In [ ]:
import os
import psycopg2
import psycopg2.extras  # Import the extras
from dotenv import load_dotenv
import vector_search

# VECTOR SEARCH WITHIN AN ALBUM
def search_vectors(query_embedding, match_threshold=0.7, match_count=10, album_id=None):
    # Load PostgreSQL full URL from .env file
    SUPABASE_DB_FULL_URL = os.getenv("SUPABASE_DB_FULL_URL")
    
    # Initialize connection and cursor
    conn = psycopg2.connect(SUPABASE_DB_FULL_URL)
    cursor = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    
    try:
        # Base SQL query
        sql_query = """
        SELECT
            vectors.id,
            vectors.content,
            vectors.media_table_id,
            1 - (vectors.embedding <=> %s::vector) AS similarity
        FROM vectors
        """
        
        # Join with album_media_link table if album_id is specified
        if album_id:
            sql_query += """
            INNER JOIN album_media_link ON vectors.media_table_id = album_media_link.media_id
            WHERE album_media_link.album_id = %s
            AND 1 - (vectors.embedding <=> %s::vector) > %s
            """
        else:
            sql_query += """
            WHERE 1 - (vectors.embedding <=> %s::vector) > %s
            """
        
        sql_query += """
        AND vectors.media_table_id IS NOT NULL
        ORDER BY similarity DESC
        LIMIT %s;
        """
        
        # Execute SQL query
        if album_id:
            cursor.execute(sql_query, (query_embedding, album_id, query_embedding, match_threshold, match_count))
        else:
            cursor.execute(sql_query, (query_embedding, query_embedding, match_threshold, match_count))
        
        # Fetch results
        results = cursor.fetchall()
        results = [dict(row) for row in results]
        
        return results

    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    finally:
        cursor.close()
        conn.close()




# Example usage
my_text = "hot dog"
my_vec = vector_search.get_embeddings(my_text)
results = search_vectors(my_vec, match_threshold=0.8, match_count=100)

for item in results:
    print(item)


In [ ]:
import os
import psycopg2
import psycopg2.extras
import openai
from openai.embeddings_utils import get_embedding, cosine_similarity
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def get_embeddings(input=""):
    """Get the vector embeddings from openAI for a given input string."""
    input = input.replace("\n", " ").strip()
    response = openai.Embedding.create(input=input, model="text-embedding-ada-002")
    embeddings = response['data'][0]['embedding']
    return embeddings

def search_vectors(query_embedding, match_threshold=0.7, match_count=10):
    SUPABASE_DB_FULL_URL = os.getenv("SUPABASE_DB_FULL_URL")
    conn = psycopg2.connect(SUPABASE_DB_FULL_URL)
    cursor = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

    try:
        # Prepare SQL query
        sql_query = """
        SELECT
            v.id, v.content, v.media_table_id,  -- Select required columns from vectors table
            m.*,  -- Select everything from media table
            1 - (v.embedding <=> %s::vector) AS similarity
        FROM vectors v
        JOIN media m ON v.media_table_id = m.id
        WHERE 1 - (v.embedding <=> %s::vector) > %s
        ORDER BY similarity DESC
        LIMIT %s;
        """

        # Execute SQL query
        cursor.execute(sql_query, (query_embedding, query_embedding, match_threshold, match_count))
        results = cursor.fetchall()

        # Process results to differentiate between vectors and media data
        formatted_results = []
        for row in results:
            vector_data = {f"vector_{key}": value for key, value in row.items() if key in ['id', 'content', 'media_table_id']}
            media_data = {f"media_{key}": value for key, value in row.items() if key not in ['id', 'content', 'media_table_id', 'embedding']}
            formatted_results.append({**vector_data, **media_data, "similarity": row['similarity']})

        return formatted_results
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    finally:
        cursor.close()
        conn.close()

# Example usage
match_threshold = 0.8
match_count = 50
my_text = "not hot dog"
my_vec = get_embeddings(my_text)
results = search_vectors(my_vec, match_threshold, match_count)
for item in results:
    print(item)


In [ ]:
# Get a scene and its keywords. 
# get all the stock media for each keyword ... Put them in the database.
# get the scene text for that scene.
# do a semantic search to finc media for that scene.
# add relavent media to the album. 

import funcs_supabase
import actions
import pexels_2
import vector_search


# def filter_dicts(dict_list, filters):
#     filtered_list = [d for d in dict_list if all(d.get(key) == value for key, value in filters.items())]
#     return filtered_list



# def vector_search_and_filter(scene_text, filters=None):
#     match_threshold = 0.6
#     match_count = 1

#     # Get embeddings for the scene text
#     my_vec = vector_search.get_embeddings(scene_text)

#     # Perform vector search
#     vec_search_results_list = vector_search.search_vectors_with_media(my_vec, match_threshold, match_count)
    
#     # Set default filter if none is provided
#     if filters is None:
#         filters = {'media_source_type': 'stock'}

#     # Initialize list to hold filtered results
#     filtered_vec_search_list = []
    
#     # Filter the search results
#     for item in vec_search_results_list:
#         # Check each filter criteria
#         if all(item.get(key) == value for key, value in filters.items()):
#             filtered_vec_search_list.append(item)

#     return filtered_vec_search_list



def get_stock_for_scene(scene_id):
    scene_data = funcs_supabase.select_data("scenes", "id", scene_id)[0]
    # print(scene_data)
    keywords = scene_data.get("theme_keywords")

    # print(keywords)
    # Downloads the images and stores in db 
    # pexels_2.process_pexels_keywords(keywords)

    album = scene_data.get("album_id")
    # print(album)

    scene_text = scene_data.get("scene_text")
    print(scene_text)

    # matching_media = vector_search_and_filter(scene_text)
    # print(matching_media)

    # for media in matching_media:
    #     media_id = media.get("media_id")
    #     # print(media_id)
    #     resp = funcs_supabase.insert_data("album_media_link", {"album_id": album, "media_id": media_id})
    #     print(resp)



def get_stock_for_project(project_id):
    print(project_id)
    scenes = actions.get_scenes_for_project(project_id)
    print(scenes)
    # for scene in scenes:
    #     print(scene)
    #     get_stock_for_scene(scene)


eh = get_stock_for_project(21)
# print(eh)



# match_threshold = 0.8
# match_count = 50
# my_text = "A happy Day"
# my_vec = get_embeddings(my_text)
# results = search_vectors(my_vec, match_threshold, match_count)




In [15]:
# Get a scene and its keywords. 
# get all the stock media for each keyword ... Put them in the database.
# get the scene text for that scene.
# do a semantic search to finc media for that scene.
# add relavent media to the album. 

import funcs_supabase
import actions
import pexels_2
import vector_search



def vector_search_and_filter(scene_text, filters=None):
    match_threshold = 0.8
    match_count = 20

    # Get embeddings for the scene text
    my_vec = vector_search.get_embeddings(scene_text)

    # Perform vector search
    vec_search_results_list = vector_search.search_vectors_with_media(my_vec, match_threshold, match_count)
    
    # Set default filter if none is provided
    if filters is None:
        filters = {'media_source_type': 'stock'}

    # Initialize list to hold filtered results
    filtered_vec_search_list = []
    
    # Filter the search results
    for item in vec_search_results_list:
        # Check each filter criteria
        if all(item.get(key) == value for key, value in filters.items()):
            filtered_vec_search_list.append(item)

    return filtered_vec_search_list



def get_stock_for_scene(scene_id, get_stock=False):
    scene_data = funcs_supabase.select_data("scenes", "id", scene_id)[0]
    # print(scene_data)
    keywords = scene_data.get("theme_keywords")

    if get_stock:
    # Downloads the images and stores in db
        pexels = pexels_2.process_pexels_keywords(keywords)
        print(pexels)

    album_id = scene_data.get("album_id")
    # print(album_id)

    scene_text = scene_data.get("scene_text")
    print(scene_text)

    matching_media = vector_search_and_filter(scene_text)
    print(matching_media)

    for media in matching_media:
        media_id = media.get("vector_media_table_id")
        # print(media_id)
        resp = funcs_supabase.insert_data("album_media_link", {"album_id": album_id, "media_id": media_id})




def get_stock_for_project(project_id, get_stock=False):
    print(project_id)
    scenes = actions.get_scenes_for_project(project_id)
    for scene in scenes:
        get_stock_for_scene(scene)


eh = get_stock_for_project(22)
# print(eh)



# match_threshold = 0.8
# match_count = 50
# my_text = "A happy Day"
# my_vec = get_embeddings(my_text)
# results = search_vectors(my_vec, match_threshold, match_count)




22


2023-10-30 21:22:13,641:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/scenes?select=%2A&project_id=eq.22&status=eq.Active "HTTP/1.1 200 OK"
2023-10-30 21:22:13,808:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/scenes?select=%2A&id=eq.172 "HTTP/1.1 200 OK"


Years passed, and every evening, under the same oak, now thick with the whispers of time, she played, her music echoing the old man's, yet imbued with her own spirit and stories.
[{'vector_id': 249, 'vector_content': 'man violin reading book listening music sitting front table musicians heritage', 'vector_media_table_id': 249, 'media_created_at': datetime.datetime(2023, 9, 12, 17, 41, 10, 389255, tzinfo=datetime.timezone.utc), 'media_description': 'a man reading a book while listening to a violin music', 'media_tags': None, 'media_custom_fields': None, 'media_url': None, 'media_type': 'video', 'media_caption': 'a man sitting in front of a table with a violin', 'media_old_filename': 'pexels-photo-6193405.jpeg', 'media_bucket': 'media', 'media_filename': '428649d5-ddab-4443-9b9b-003076d7f35f.jpeg', 'media_media_data': {'file_size': 89238, 'file_type': 'jpeg', 'dimensions': {'width': 1200, 'height': 630, 'channels': 3}, 'mean_color': [69.18527645502645, 92.29996296296297, 116.297898148148

2023-10-30 21:22:18,364:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:22:18,465:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:22:19,125:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:22:19,296:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:22:19,420:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:22:19,824:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:22:20,267:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:22:20,883:INF

In a forgotten town, beneath the shadow of an ancient oak, an old man played a violin, its notes weaving through the twilight like threads of gold.
[{'vector_id': 203, 'vector_content': 'a violin leaning against a log in a field', 'vector_media_table_id': 203, 'media_created_at': datetime.datetime(2023, 9, 12, 17, 36, 44, 872227, tzinfo=datetime.timezone.utc), 'media_description': 'a violin leaning on a log of wood', 'media_tags': None, 'media_custom_fields': None, 'media_url': None, 'media_type': 'video', 'media_caption': 'a violin leaning against a log in a field', 'media_old_filename': 'autumn-grass-violin-wind-5721976.jpeg', 'media_bucket': 'media', 'media_filename': '56cd6c2f-0125-4763-bc73-d287380d5fd0.jpeg', 'media_media_data': {'file_size': 118402, 'file_type': 'jpeg', 'dimensions': {'width': 1200, 'height': 630, 'channels': 3}, 'mean_color': [127.66202380952382, 141.55075925925925, 155.81245767195767, 0.0]}, 'media_thumbnail_url': 'https://fpyltvtkpkrkzortucoa.supabase.co/stor

...

2023-10-30 21:22:51,473:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:22:51,752:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:22:51,918:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:22:52,281:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:22:52,558:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:22:52,842:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:22:53,087:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:22:53,369:INF

And so, the violin continued to sing its timeless songs, bridging generations under the watchful eye of the ancient oak.


2023-10-30 21:22:58,865:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:22:58,943:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"


[{'vector_id': 203, 'vector_content': 'a violin leaning against a log in a field', 'vector_media_table_id': 203, 'media_created_at': datetime.datetime(2023, 9, 12, 17, 36, 44, 872227, tzinfo=datetime.timezone.utc), 'media_description': 'a violin leaning on a log of wood', 'media_tags': None, 'media_custom_fields': None, 'media_url': None, 'media_type': 'video', 'media_caption': 'a violin leaning against a log in a field', 'media_old_filename': 'autumn-grass-violin-wind-5721976.jpeg', 'media_bucket': 'media', 'media_filename': '56cd6c2f-0125-4763-bc73-d287380d5fd0.jpeg', 'media_media_data': {'file_size': 118402, 'file_type': 'jpeg', 'dimensions': {'width': 1200, 'height': 630, 'channels': 3}, 'mean_color': [127.66202380952382, 141.55075925925925, 155.81245767195767, 0.0]}, 'media_thumbnail_url': 'https://fpyltvtkpkrkzortucoa.supabase.co/storage/v1/object/public/media/56cd6c2f-0125-4763-bc73-d287380d5fd0.jpeg', 'media_source': 'Pexels', 'media_source_data': {'id': 5721976, 'url': 'https:

2023-10-30 21:22:59,061:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:22:59,317:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:22:59,607:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:23:00,442:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:23:00,741:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:23:01,160:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:23:01,427:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-10-30 21:23:01,748:INF